# Calculation 05 - Surface energies
The following notebook will calculate the surface energies of Nickel-based EAM and MEAM potentials. The first cell needs updating depending on the potential file you are using and the compound you want to run the calculation for.\
At the end of the notebook, there is a "fast run" which will allow you to run this notebook for all potentials and all surfaces in one go.

## 📌 Specify filenames and element related data
For any potential, please specify the filename of the potential file as follows:\
`potentials/filename.eam`\
`potentials/filename.eam.fs`\
`potentials/filename.eam.alloy`\
`potentials/filename.meam`

Then uncomment the respective lines, depending on which compound you want to run the calculation for.

**Note:** to call Non-LAMMPS variables within a LAMMPS command, please use an f-string. To call LAMMPS variables, instead of using an f-string, just use curly braces with a leading dollar sign - like this: `${x}`.

In [ ]:
potentialpath = "potentials/Ni1986.eam"
latticeconstant = 3.52
equilibriumlatticeconstant = 3.52
surface = "100"
outputfile = "data/05_surface-energies_single.dat"

## ⬇️ Import PyLammps module

In [ ]:
from lammps import PyLammps
lmp = PyLammps()

lmp.clear()

## ⬇️ Add parameters for MEAM potentials
There is no need to change the following cell. It defines two parameters that are necessary for reading in MEAM potentials.\
**Please do note that the libraryelements variable has to contain the element symbols in the same order as they appear in the library file!**

In [ ]:
if "meam" in potentialpath:
    libraryfile = str(potentialpath.replace('.meam', '.library.meam'))
    
    if "NiAlCo2" in potentialpath:
        libraryelements = "Ni Al Co"
    elif "NiAl2" in potentialpath:
        libraryelements = "Al Ni"
    else:
        libraryelements = "Ni"

## ⬇️ Initialize simulation I
`units` sets the unit system for this simulation. "metal" units are Angstroms for distance, eV for energy, etc.\
`dimension` is self-explanatory and can either be 2 or 3.\
`boundary` sets the boundary conditions in x-, y- and z-direction. Here, they are periodic in all three dimensions.\
`atom_style` specifies additional attributes depending on the style - "atomic" doesn't have any.

In [ ]:
lmp.units('metal')
lmp.dimension('3')
lmp.boundary('p p p')
lmp.atom_style('atomic')

## ⬇️ Create atoms I
`lattice` specifies the predefined or customized lattice and the respective lattice constant.\
`region` specifies the simulation cell as a geometric shape, like a sphere, cylinder or block. The default coordinate unit is a "lattice parameter", meaning each coordinate is an integer factor times the lattice parameter specified in the `lattice` command. "cell" is the user-assigned ID for this specific region, but could also be called "1", "a" or anything else. For periodic boundary conditions, the cell dimensions should be multiples of the lattice constant to prevent unwanted overlaps (as explained in the documentation for the `create_atoms` command).\
`create_box` actually creates the simulation cell as defined in the `region` command. The integer defines the number of element types that will be used, the string defines the region in which the cell will be created.\
`create_atoms` adds atoms to the lattice within a specific region, so this has to happen after defining and creating the simulation box. The integer defines the number of element types used.\
`replicate` would allow to change the size of the simulation, and in this position is equivalent to changing the coordinates in the `region` command.
`mass` does not need to specified for any EAM or MEAM potential since it is used from the potential file itself.

In [ ]:
if surface == '100':    
    lmp.variable(f'x equal {equilibriumlatticeconstant}*5')
    lmp.variable(f'y equal {equilibriumlatticeconstant}*5')
    lmp.variable(f'z equal {equilibriumlatticeconstant}*5')
    
    lmp.lattice(f'fcc {latticeconstant}')
    lmp.region('cell block 0 ${x} 0 ${y} 0 ${z} units box')
    lmp.create_box('1 cell')
    lmp.create_atoms('1 box')
    
elif surface == '110':
    lmp.variable(f'x equal {equilibriumlatticeconstant}*5')
    lmp.variable(f'y equal {equilibriumlatticeconstant}*7*sqrt(2)/2')
    lmp.variable(f'z equal {equilibriumlatticeconstant}*7*sqrt(2)/2')

    lmp.lattice(f'fcc {latticeconstant} orient x 0 0 1 orient y 1 1 0 orient z -1 1 0')
    lmp.region('cell block 0 ${x} 0 ${y} 0 ${z} units box')
    lmp.create_box('1 cell')
    lmp.create_atoms('1 box')
    
elif surface == '111':
    lmp.variable(f'x equal {equilibriumlatticeconstant}*8*sqrt(6)/4')
    lmp.variable(f'y equal {equilibriumlatticeconstant}*7*sqrt(2)/2')
    lmp.variable(f'z equal {equilibriumlatticeconstant}*9*sqrt(3)/3')

    lmp.lattice(f'fcc {latticeconstant} orient x 1 1 2 orient y -1 1 0 orient z -1 -1 1')
    lmp.region('cell block 0 ${x} 0 ${y} 0 ${z} units box')
    lmp.create_box('1 cell')
    lmp.create_atoms('1 box')

## ⬇️ Read in potential file I
`pair_style` specifies what kind of interatomic potential will be used.\
`pair_coeff` specifies the potential file to be read in. The input parameters change depending on the potential style. that the pair potential coefficients are stored in. The first two integers define the force field coefficients between atom pairs. The asterisks include all atom types.\
`neighbor` and `neigh_modify` would set parameters for the neighbor lists - the default parameters when the commands are not used can be found in the LAMMPS documentation.

In [ ]:
if "meam" in potentialpath:
    lmp.pair_style('meam')
    lmp.pair_coeff(f'* * {libraryfile} {libraryelements} {potentialpath} Ni')
elif "eam.alloy" in potentialpath:
    lmp.pair_style('eam/alloy')
    lmp.pair_coeff(f'* * {potentialpath} Ni')
elif "eam.fs" in potentialpath:
    lmp.pair_style('eam/fs')
    lmp.pair_coeff(f'* * {potentialpath} Ni')
else:
    lmp.pair_style('eam')
    lmp.pair_coeff(f'* * {potentialpath}')

## ⬇️ Define computes I
`compute`s define "formulas" that can later on be computed at each timestep of e.g. a thermodynamic run, minimization or dump output.\
The first one allows to store the potential energy for each atom during any "run".\
The second one uses the first one, summing up the potential energies of all atoms in the system.\
The third one allows to calculate the centro-symmetry parameter.

In [ ]:
lmp.compute('Epot all pe/atom')
lmp.compute('Epotall all reduce sum c_Epot')
lmp.compute('Csym all centro/atom fcc')

## ⬇️ Define thermo settings I
`min_style` would set which minimization algorithm should be used for the minimization. The default value is `cg` - conjugate gradient.\
`thermo` sets the timestep interval at which computes shall be performed and printed during the run.\
`thermo_style` allows to specify what information shall be printed at each of those timesteps - in this case, it is customized to calculate and print the timestep, the length of the simulation box in x, y and z direction, and the total potential energy of the system.

In [ ]:
lmp.thermo(10)
lmp.thermo_style('custom step lx ly lz pe c_Epotall')

## ⬇️ Define dump file I
`dump` will print the defined properties in a dump file at the desired timestep interval. With these files, the simulation can be visualized using software like OVITO.

In [ ]:
lmp.dump('start all cfg 1000 dump/05-'+str(surface)+'_start.*.cfg mass type xs ys zs c_Csym c_Epot')
lmp.dump_modify('start element Ni')

## ⬇️ Run minimization I
`minimize`s the energy of the simulation cell, printing out the desired information (as specified in `thermo_style`) at every `thermo(10)`th timestep.

In [ ]:
lmp.minimize('1e-10 1e-10 10000 100000')

## ⬇️ Define variable formulas I
This section defines a `variable` by assigning a variable to a formula that is not yet evaluated.\
This one is the total energy of the simulation cell E.

In [ ]:
lmp.variable('E equal "c_Epotall"')

In [ ]:
lmp.variable('N equal "count(all)"')

## ⬇️ Calculate properties I
Here, the formula defined above is calculated with the current value of the formula variable and stored in a new variable.

In [ ]:
lmp.variable('Ei equal $E')

In [ ]:
lmp.variable('Ni equal $N')
lmp.variable('E_coh equal ${Ei}/${Ni}')

## ⬇️ Undump I
Close dump file.

In [ ]:
lmp.undump('start')

## ⬇️ Clear all LAMMPS input
To generate a free surface, the boundary conditions have to be changed. To do so, nearly all input parameters need to be redefined.

In [ ]:
lmp.clear()

## ⬇️ Initialize simulation II
`units` sets the unit system for this simulation. "metal" units are Angstroms for distance, eV for energy, etc.\
`dimension` is self-explanatory and can either be 2 or 3.\
`boundary` sets the boundary conditions in x-, y- and z-direction. Here, they are periodic in the x and y dimensions, but non-periodic and shrink-wrapped in the z dimension.\
`atom_style` specifies additional attributes depending on the style - "atomic" doesn't have any.

In [ ]:
lmp.units('metal')
lmp.dimension('3')
lmp.boundary('p p m')
lmp.atom_style('atomic')

## ⬇️ Create atoms II
`lattice` specifies the predefined or customized lattice and the respective lattice constant.\
`region` specifies the simulation cell as a geometric shape, like a sphere, cylinder or block. The default coordinate unit is a "lattice parameter", meaning each coordinate is an integer factor times the lattice parameter specified in the `lattice` command. "cell" is the user-assigned ID for this specific region, but could also be called "1", "a" or anything else. For periodic boundary conditions, the cell dimensions should be multiples of the lattice constant to prevent unwanted overlaps (as explained in the documentation for the `create_atoms` command).\
`create_box` actually creates the simulation cell as defined in the `region` command. The integer defines the number of element types that will be used, the string defines the region in which the cell will be created.\
`create_atoms` adds atoms to the lattice within a specific region, so this has to happen after defining and creating the simulation box. The integer defines the number of element types used.\
`replicate` would allow to change the size of the simulation, and in this position is equivalent to changing the coordinates in the `region` command.
`mass` does not need to specified for any EAM or MEAM potential since it is used from the potential file itself.

In [ ]:
if surface == '100':    
    lmp.variable(f'x equal {equilibriumlatticeconstant}*5')
    lmp.variable(f'y equal {equilibriumlatticeconstant}*5')
    lmp.variable(f'z equal {equilibriumlatticeconstant}*5')
    
    lmp.lattice(f'fcc {latticeconstant}')
    lmp.region('cell block 0 ${x} 0 ${y} 0 ${z} units box')
    lmp.create_box('1 cell')
    lmp.create_atoms('1 box')
    
elif surface == '110':
    lmp.variable(f'x equal {equilibriumlatticeconstant}*5')
    lmp.variable(f'y equal {equilibriumlatticeconstant}*7*sqrt(2)/2')
    lmp.variable(f'z equal {equilibriumlatticeconstant}*7*sqrt(2)/2')

    lmp.lattice(f'fcc {latticeconstant} orient x 0 0 1 orient y 1 1 0 orient z -1 1 0')
    lmp.region('cell block 0 ${x} 0 ${y} 0 ${z} units box')
    lmp.create_box('1 cell')
    lmp.create_atoms('1 box')
    
elif surface == '111':
    lmp.variable(f'x equal {equilibriumlatticeconstant}*8*sqrt(6)/4')
    lmp.variable(f'y equal {equilibriumlatticeconstant}*7*sqrt(2)/2')
    lmp.variable(f'z equal {equilibriumlatticeconstant}*9*sqrt(3)/3')

    lmp.lattice(f'fcc {latticeconstant} orient x 1 1 2 orient y -1 1 0 orient z -1 -1 1')
    lmp.region('cell block 0 ${x} 0 ${y} 0 ${z} units box')
    lmp.create_box('1 cell')
    lmp.create_atoms('1 box')

## ⬇️ Read in potential file II
`pair_style` specifies what kind of interatomic potential will be used.\
`pair_coeff` specifies the potential file to be read in. The input parameters change depending on the potential style. that the pair potential coefficients are stored in. The first two integers define the force field coefficients between atom pairs. The asterisks include all atom types.\
`neighbor` and `neigh_modify` would set parameters for the neighbor lists - the default parameters when the commands are not used can be found in the LAMMPS documentation.

In [ ]:
if "meam" in potentialpath:
    lmp.pair_style('meam')
    lmp.pair_coeff(f'* * {libraryfile} {libraryelements} {potentialpath} Ni')
elif "eam.alloy" in potentialpath:
    lmp.pair_style('eam/alloy')
    lmp.pair_coeff(f'* * {potentialpath} Ni')
elif "eam.fs" in potentialpath:
    lmp.pair_style('eam/fs')
    lmp.pair_coeff(f'* * {potentialpath} Ni')
else:
    lmp.pair_style('eam')
    lmp.pair_coeff(f'* * {potentialpath}')

## ⬇️ Define computes II
`compute`s define "formulas" that can later on be computed at each timestep of e.g. a thermodynamic run, minimization or dump output.\
The first one allows to store the potential energy for each atom during any "run".\
The second one uses the first one, summing up the potential energies of all atoms in the system.\
The third one allows to calculate the centro-symmetry parameter.

In [ ]:
lmp.compute('Epot all pe/atom')
lmp.compute('Epotall all reduce sum c_Epot')
lmp.compute('Csym all centro/atom fcc')

## ⬇️ Define thermo settings II
`min_style` would set which minimization algorithm should be used for the minimization. The default value is `cg` - conjugate gradient.\
`thermo` sets the timestep interval at which computes shall be performed and printed during the run.\
`thermo_style` allows to specify what information shall be printed at each of those timesteps - in this case, it is customized to calculate and print the timestep, the length of the simulation box in x, y and z direction, and the total potential energy of the system.

In [ ]:
lmp.thermo(10)
lmp.thermo_style('custom step lx ly lz pe c_Epotall')

## ⬇️ Define dump file II
`dump` will print the defined properties in a dump file at the desired timestep interval. With these files, the simulation can be visualized using software like OVITO.

In [ ]:
lmp.dump('stop all cfg 1000 dump/05-'+surface+'_stop.*.cfg mass type xs ys zs c_Csym c_Epot')
lmp.dump_modify('stop element Ni')

## ⬇️ Run minimization II
`minimize`s the energy of the simulation cell, printing out the desired information (as specified in `thermo_style`) at every `thermo(10)`th timestep.

In [ ]:
lmp.minimize('1e-10 1e-10 10000 100000')

## ⬇️ Define variable formulas II
This section defines some `variable`s by assigning variables to a formula that are not yet evaluated.\
The first one is the total energy of the simulation cell E, the second one the area of the stacking fault plane. 

In [ ]:
lmp.variable('E equal "c_Epotall"')
lmp.variable('N equal "count(all)"')
lmp.variable('A equal "lx*ly*1e-20"')

## ⬇️ Calculate properties II
Again, the formula defined above is calculated with the current value of the formula variable and stored in a new variable.
A second variable, calculating the stacking fault energy, is defined and will be calculated in the `print` command with the current variable values.

In [ ]:
lmp.variable('Ef equal $E')
lmp.variable('Nf equal $N')

lmp.variable('J_to_eV equal 1.60217657e-16')
lmp.variable('Es equal ((${Ef}-${Ei})*${J_to_eV})/(2*${A})')

## ⬇️ Undump II
Close dump file.

In [ ]:
lmp.undump('stop')

## ⬇️ Print desired output
This section `print`s these values to the output file, for which you chose a filename in the beginning.

In [ ]:
potentialname = potentialpath.replace('potentials/', '')
lmp.print(f'"{potentialname}, {surface}'+', ${E_coh}, ${Ni}, ${Nf}, ${Ei}, ${Ef}, ${Es}" append '+f'{outputfile} screen no')

🏁**ALL DONE!**🏁

🏁**FAST RUN DONE!**🏁